In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os
import pandas as pd

In [4]:
def setup(url):
    driver = webdriver.Chrome()
    driver.get(url)
    return driver

def teardown(driver):
    driver.quit()

def save_data_to_file(data, filename):
    if not os.path.exists('data'):
        os.makedirs('data')
    
    with open(f'data/{filename}', 'w', encoding='utf-8') as file:
        file.write(data)

In [ ]:
Discussions_urls=[]
def  get_discussing_urls():
    driver=setup("https://community.shopify.com/c/shopify-discussion/ct-p/shopify-discussion")
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.more-boards")))
    load_all=driver.find_elements(By.CSS_SELECTOR,"button.more-boards")
    
    while True:
       if len(load_all)>0:
           load_all[0].click()
           try:
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.more-boards")))
                load_all=driver.find_elements(By.CSS_SELECTOR,"button.more-boards")
           except:
               load_all=[]
       else:
           break
    
    links_list=driver.find_elements(By.CSS_SELECTOR,".tiled-navigation-boards>a")
    for i,url in enumerate(links_list[:-1]):
        
        Discussions_urls.append(url.get_attribute("href"))

   
        
get_discussing_urls()

df=pd.DataFrame(data={"Discussions_urls":Discussions_urls});
df.to_csv("Discussions_urls_data.csv")  

# print(discussing_urls)


In [ ]:
single_url=[]
def scraper(url):
    
    driver = setup(url)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pagination-div>a")))
    page_counts = driver.find_elements(By.CSS_SELECTOR,"div.pagination-div>a")
    last_page=page_counts[-2].text #to get count of number of pages

    
    for page_no in range(1, last_page):
        try:
            if page_no == 1:
                data = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.res-data")))
            else:
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pagination-div>a")))
                pagination = driver.find_elements(By.CSS_SELECTOR,"div.pagination-div>a")            
                driver.implicitly_wait(5)
                pagination[-1].click()
                data = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.res-data")))
        except Exception as e:
            print("Out of boundary index. Your all url's are fetched. Wait for execution")
            continue
            
        
        for i, ele in enumerate(data):
            try:
                data = driver.find_elements(By.CSS_SELECTOR, "div.res-data")
                divs=data[i]                
                link=divs.find_element(By.CSS_SELECTOR,"div.res-subject > a")
                post_url = link.get_attribute("href")
                single_url.append(post_url)            
            
            except Exception as e:
                print(f"some error occur. But url's are fetched.")
    
    # print(len(single_url))    
    teardown(driver)
    
    
df = pd.read_csv('discussing_urls_data.csv')
Discussions_urls_from_csv = df["discussing_urls"].tolist()

for i,link in enumerate(Discussions_urls_from_csv): 
    scraper(link)


df=pd.DataFrame(data={"url":single_url});
df.to_csv("post_urls.csv")  
    

In [ ]:
df = pd.read_csv('post_urls.csv')
urls = df["url"].tolist()

def data_pages():
    for i,link in enumerate(urls):
        # change number acc. to where you pause the last execution. Put last printed number in "if" condition to resume.
        # if i<=2105: 
        #     continue;
        # else:
            driver = setup(link)
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.lia-quilt-row-main")))
            post_data = element.get_attribute("outerHTML")        
            save_data_to_file(post_data, f'page_{i+1}.html')
            print(i)        
            teardown(driver)
        
data_pages()

In [ ]:
df = pd.read_csv('post_urls.csv')
urls = df["url"].tolist()

d={"url":urls,"title":[],"user_name":[],"user_title":[],"user_profile_link":[],"number_of_posts_by_user":[],"number_of_solutions_by_user":[],"number_of_kudos_by_user":[],"timestamp":[],"post_content":[],"solved":[],"labels_list":[],"number_of_views":[],"number_of_likes":[],"replies":[],"accepted_sol":[]}

for i,file in enumerate(os.listdir("data")):
    with open(f"data/{file}",encoding="utf8") as f:
        html_doc=f.read()
    soup=BeautifulSoup(html_doc,"html.parser")    
    
    title=soup.find("h1",class_="page-title")
    if title==None:
        continue
    d["title"].append(title.text)
    
    user_name=soup.find("a",class_="lia-user-name-link")
    d["user_name"].append(user_name.text)
    
    user_profile_link=user_name.get_attribute_list("href")
    d["user_profile_link"].append(user_profile_link[0])
   
    user_title_container=soup.find("span",class_="lia-message-unread")
    user_title=user_title_container.find_next("div")    
    d["user_title"].append(user_title.text.strip())
    
    number_of_posts_by_user=soup.find(class_="lia-message-author-posts-count")
    d["number_of_posts_by_user"].append(number_of_posts_by_user.text)
    
    number_of_solutions_by_user=soup.find(class_="lia-message-author-solutions-count")
    d["number_of_solutions_by_user"].append(number_of_solutions_by_user.text.strip())
    
    nunber_of_kudos_by_user=soup.find(class_="lia-message-author-kudos-count")
    d["number_of_kudos_by_user"].append(nunber_of_kudos_by_user.text.strip())
    
    publish_date=soup.select(".DateTime > meta")[0].get_attribute_list("content")
    d["timestamp"].append(publish_date[0])
    
    post_content_data=soup.find(class_="lia-message-body-content")
    for content in post_content_data(['style', 'script']):
        content.decompose().strip()
    post_content=' '.join(post_content_data.stripped_strings)
    d["post_content"].append(post_content)    
    
    if soup.find(class_="lia-panel-feedback-banner-safe"):
        solve="Y"
        accepted_sol=soup.find_all(class_="lia-message-body-content")
        for content in accepted_sol[1](['style', 'script']):
                content.decompose().strip()
        sol_content=' '.join(accepted_sol[1].stripped_strings)

    else:
        solve="N"
        sol_content=" "
    d["solved"].append(solve)
    d["accepted_sol"].append(sol_content)
    
    label_data=soup.find_all(class_="label")
    labels=[]
    for label_arr in label_data:
        labels.append((label_arr.text).strip())
    d["labels_list"].append(labels)
        
    view_count=soup.find(class_="lia-message-stats-count")
    if view_count:
        d["number_of_views"].append(view_count.text.strip())   
    else:
        d["number_of_views"].append("0")   
  
    
    like_count=soup.find(class_="MessageKudosCount")
    d["number_of_likes"].append(like_count.text.strip())  
    
    replies=soup.find(class_="lia-threaded-detail-display-message-view")
    reply_list=[]
    if replies:
        reply_data=soup.find_all(class_="lia-message-body-content")
        for content in reply_data[1](['style', 'script']):
            content.decompose().strip()
        reply_content=' '.join(reply_data[1].stripped_strings)
        reply_list.append(reply_content)
    else:
        pass
    d["replies"].append(reply_list)
    
    
  
    
df=pd.DataFrame(data=d);
df.to_csv("shopify_discussion_data.csv")    